In [ ]:
# Canonical vgs-check helper using package parser
from eesizer.analysis.oplog import parse_vgs_vth_from_oplog
import json
import os

def vgscheck_from_oplog_file(op_txt_path, output_txt_path=None):
    """Read an ngspice op.txt, parse vgs/vth with the centralized parser,
    write a small JSON summary and optional plain-text summary for backward compatibility.
    Returns: list of dicts [{name, vgs, vth, margin}, ...]
    """
    with open(op_txt_path, "r", encoding="utf-8") as f:
        text = f.read()
    devices = parse_vgs_vth_from_oplog(text)
    summary = []
    for d in devices:
        summary.append({"name": d.name, "vgs": d.vgs, "vth": d.vth, "margin": d.margin})
    # write JSON summary next to the op file
    json_path = os.path.splitext(op_txt_path)[0] + "_summary.json"
    try:
        with open(json_path, "w", encoding="utf-8") as j:
            json.dump(summary, j, indent=2)
    except Exception:
        pass
    # optional plain-text summary
    if output_txt_path:
        try:
            with open(output_txt_path, "w", encoding="utf-8") as out:
                for s in summary:
                    out.write(f"{s['name']}: vgs={s['vgs']}, vth={s['vth']}, margin={s['margin']}\n")
        except Exception:
            pass
    return summary

# Backwards-compatible wrappers for the old brittle pipeline. These now delegate to the
# canonical parser (no CSV intermediate files) so all existing call-sites keep working.
# We'll remove these wrappers in a follow-up once all call-sites are updated.
import warnings

def filter_lines(input_file, output_file):
    warnings.warn("filter_lines is deprecated; delegating to vgscheck_from_oplog_file", DeprecationWarning)
    # assume input_file is op.txt path
    try:
        vgscheck_from_oplog_file(input_file, output_file)
    except Exception:
        # ensure downstream cells still see an output file
        open(output_file, "w").close()


def convert_to_csv(input_txt, output_csv):
    warnings.warn("convert_to_csv is deprecated; no-op delegating to parser", DeprecationWarning)
    # create an empty csv placeholder if needed
    try:
        open(output_csv, "w").close()
    except Exception:
        pass


def format_csv_to_key_value(input_csv, output_txt):
    warnings.warn("format_csv_to_key_value is deprecated; delegating to parser", DeprecationWarning)
    # Try to locate sibling op.txt if the pipeline used intermediate names
    maybe_op = os.path.splitext(input_csv)[0] + ".txt"
    if os.path.exists(maybe_op):
        try:
            vgscheck_from_oplog_file(maybe_op, output_txt)
            return
        except Exception:
            pass
    # fallback: treat input_csv as the op text file
    try:
        vgscheck_from_oplog_file(input_csv, output_txt)
    except Exception:
        # create fallback empty file
        open(output_txt, "w").close()


In [ ]:
# Robust vgs/vth parser and compatibility writer
# Replaces the brittle filter_lines -> convert_to_csv -> format_csv_to_key_value pipeline.
# Uses the package parser and writes both JSON and a simple text summary for backward compatibility.
from eesizer.analysis.oplog import parse_vgs_vth_from_oplog
import json
import os


def vgscheck_from_oplog_file(op_txt_path: str, output_txt_path: str):
    """Parse op.txt and write a JSON summary and a simple text summary.

    Returns a list of dicts: [{name, vgs, vth, margin}, ...].
    This keeps downstream notebook cells working while we centralize parsing.
    """
    text = ""
    if os.path.exists(op_txt_path):
        try:
            with open(op_txt_path, "r", encoding="utf-8") as f:
                text = f.read()
        except Exception:
            text = ""
    biases = parse_vgs_vth_from_oplog(text)

    summary = []
    for b in biases:
        summary.append({
            "name": b.name,
            "vgs": b.vgs,
            "vth": b.vth,
            "margin": b.margin,
        })

    # write machine-readable JSON summary next to the provided text path
    try:
        json_path = os.path.splitext(output_txt_path)[0] + "_summary.json"
        with open(json_path, "w", encoding="utf-8") as jf:
            json.dump(summary, jf, indent=2)
    except Exception:
        pass

    # write a simple text summary for backward compatibility (one device per line)
    try:
        with open(output_txt_path, "w", encoding="utf-8") as tf:
            if not summary:
                tf.write("")
            else:
                for d in summary:
                    # format: NAME vgs=... vth=... margin=...
                    tf.write(f"{d['name']} vgs={d['vgs']} vth={d['vth']} margin={d['margin']}\n")
    except Exception:
        pass

    return summary


# Provide tiny shims so older call-sites that call the three-step pipeline will still work
# but now they delegate directly to the robust parser above.

def filter_lines(input_file, output_file):
    # compatibility: ignore filtering step and let parser handle extraction
    # keep file I/O minimal to avoid breaking callers that expect output_file to exist
    try:
        if os.path.exists(input_file):
            with open(input_file, "r", encoding="utf-8") as fr:
                data = fr.read()
        else:
            data = ""
        # write the raw op.txt content to output_file so downstream code that expects
        # a filtered file still sees something (not used by new parser)
        with open(output_file, "w", encoding="utf-8") as fw:
            fw.write(data)
    except Exception:
        # best-effort: create an empty file
        try:
            open(output_file, "w", encoding="utf-8").close()
        except Exception:
            pass


def convert_to_csv(input_txt, output_csv):
    # compatibility no-op: the robust parser reads the original op.txt directly.
    # create an empty csv so callers that inspect the path don't fail.
    try:
        open(output_csv, "w", encoding="utf-8").close()
    except Exception:
        pass


def format_csv_to_key_value(input_csv, output_txt):
    # compatibility: derive op.txt path from input_csv (best-effort) and call the new helper
    # If a real op.txt exists next to input_csv, prefer it; otherwise treat input_csv as op text.
    # Attempt to find an op.txt sibling file in the same directory.
    input_dir = os.path.dirname(input_csv) or "."
    possible_op = os.path.join(input_dir, "op.txt")
    if os.path.exists(possible_op):
        op_path = possible_op
    else:
        # fallback: assume input_csv actually contains op-style text
        op_path = input_csv

    return vgscheck_from_oplog_file(op_path, output_txt)


In [ ]:
# Compatibility wrappers for removed CSV pipeline
# These provide the old function names so existing notebook call-sites keep working.
from eesizer.analysis.oplog import parse_vgs_vth_from_oplog
import os

def filter_lines(input_file, output_file):
    """Lightweight pass-through: copy input to output so older call-sites that expect
    an intermediate filtered file continue to work."""
    try:
        with open(input_file, 'r') as f:
            data = f.read()
    except Exception:
        data = ''
    with open(output_file, 'w') as f:
        f.write(data)
    return output_file


def convert_to_csv(input_txt, output_csv):
    """No-op conversion: write the same text to output_csv. The downstream formatter
    will parse op-log content if present."""
    try:
        with open(input_txt, 'r') as f:
            data = f.read()
    except Exception:
        data = ''
    with open(output_csv, 'w') as f:
        f.write(data)
    return output_csv


def format_csv_to_key_value(input_csv, output_txt):
    """Produce a key=value summary file from an op-log-like text. Uses
    parse_vgs_vth_from_oplog for robust extraction.
    """
    text = ''
    try:
        with open(input_csv, 'r') as f:
            text = f.read()
    except Exception:
        text = ''

    devices = parse_vgs_vth_from_oplog(text)

    # If parsing returned nothing, try a sibling op.txt in the same directory
    if not devices:
        dirn = os.path.dirname(input_csv) or '.'
        op_path = os.path.join(dirn, 'op.txt')
        if os.path.exists(op_path):
            try:
                with open(op_path, 'r') as f:
                    op_text = f.read()
                devices = parse_vgs_vth_from_oplog(op_text)
            except Exception:
                devices = []

    # Write a simple key=value per-line summary that older notebook code expects
    try:
        with open(output_txt, 'w') as f:
            for d in devices:
                # device object may be a simple namespace/dataclass
                name = getattr(d, 'name', str(d))
                vgs = getattr(d, 'vgs', '')
                vth = getattr(d, 'vth', '')
                margin = getattr(d, 'margin', '')
                f.write(f"name={name},vgs={vgs},vth={vth},margin={margin}\n")
    except Exception:
        # best-effort: create an empty file so callers don't break
        open(output_txt, 'w').close()

    return output_txt


In [ ]:
# Replace in-notebook brittle tool_calling/optimization with orchestrator-backed wrappers
from typing import Dict, Any, Optional
from eesizer.agents.orchestrator import Orchestrator


def tool_calling(netlist_text: str, tool_chain: Dict[str, Any], run_dir: Optional[str] = None) -> Dict[str, Any]:
    """Run the provided tool_chain against the netlist using the Orchestrator.

    Returns the orchestrator's results dict. This replaces the old CSV-based pipeline and
    ensures the notebook uses the package-level parser and simulator.
    """
    orch = Orchestrator(run_dir=run_dir)
    return orch.run_once(netlist_text, tool_chain)


def optimization(netlist_text: str, tool_chain: Dict[str, Any], run_dir: Optional[str] = None) -> Dict[str, Any]:
    """Simple optimization wrapper that delegates a single-step evaluation to the Orchestrator.

    The notebook previously implemented a multi-step optimization loop inline. For now we
    provide a lightweight replacement that performs one evaluation and returns the summary
    so the rest of the notebook (which expects a dict) continues to work. We can iterate
    and expand this into a full optimizer in the package (recommended) later.
    """
    # Single evaluation using orchestrator
    results = tool_calling(netlist_text, tool_chain, run_dir=run_dir)
    return {"optimization_result": results}


In [4]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import openai
import requests
import re
import boto3
import json
import os
import csv
import time
import shutil
import pandas as pd
from openai import OpenAI
from scipy.fft import fft, fftfreq

In [5]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API URL and API key
#api_base_url = os.getenv('API_URL')
api_key = os.getenv('OPENAI_API_KEY')

#print(f"API URL: {api_base_url}")
#print(f"API Key: {api_key}")

In [6]:
from eesizer.llm.base import make_chat_completion_request, make_chat_completion_request_function
# LLM wrapper functions are provided by the eesizer.llm package to keep notebook cells thin and testable.

In [7]:
# make_chat_completion_request_function is provided by eesizer.llm.base
# imported at the top-level to keep notebook cells minimal

## User Input

In [8]:
tasks_generation_question = " This is a circuit netlist, optimize this circuit with a output swing above 1.2V, input offset smaller than 0.001V, input common mode range bigger than 1.2, ac gain and transient gain above 65dB, unity bandwidth above 5000000Hz, phase margin bigger than 45 degree, power smaller than 0.05W, cmrr bigger than 100dB and thd small than -26dB"
netlist = ''' 
.title Basic amp
.include 'ptm_90.txt'
.param Vcm = 0.6
M3 d3 in2 midp vdd pmos W=1u L=90n
M4 d4 in1 midp vdd pmos W=1u L=90n
M6 midp bias1 vdd vdd pmos W=1u L=90n

M1 d1 in1 midn 0 nmos W=1u L=90n
M2 d2 in2 midn 0 nmos W=1u L=90n
M5 midn bias2 0 0 nmos W=1u L=90n

M7 d1 g7 vdd vdd pmos W=1u L=90n
M8 d2 g7 vdd vdd pmos W=1u L=90n
M9 g7 bias3 d1 vdd pmos W=1u L=90n
M10 d10 bias3 d2 vdd pmos W=1u L=90n

M15 g7 bias5 s15 0 nmos W=1u L=90n
M16 s15 bias6 g7 vdd pmos W=1u L=90n
M17 d10 bias5 s17 0 nmos W=1u L=90n
M18 s17 bias6 d10 vdd pmos W=1u L=90n

M11 s15 bias4 d4 0 nmos W=1u L=90n
M12 s17 bias4 d3 0 nmos W=1u L=90n
M13 d4 s15 0 0 nmos W=1u L=90n
M14 d3 s15 0 0 nmos W=1u L=90n

M19 out d10 vdd vdd pmos W=1u L=90n
M20 out s17 0 0 nmos W=1u L=90n

Cc1 out d10 5p
Cc2 out s17 5p
Cl out 0 10p 
Rl out 0 1k

vbias1 bias1 0 DC 0.6
vbias2 bias2 0 DC 0.6
vbias3 bias3 0 DC 0.6
vbias4 bias4 0 DC 0.6
vbias5 bias5 0 DC 0.6
vbias6 bias6 0 DC 0.6

vdd vdd 0 1.2

Vcm cm 0 DC {Vcm}
Eidp cm in1 diffin 0 1
Eidn cm in2 diffin 0 -1
Vid diffin 0 AC 1 SIN (0 1u 10k 0 0)
.op

.end

'''

## Task Decpmposition

In [9]:
## prompt for tasks generation based on user input
tasks_generation_template = f""" 

You are an expert at analogue circuit design. You are required to decomposit tasks from initial user input. Here are some examples:

# Example 1
question = '''
.title Basic Amplifier
Vdd vdd 0 1.8V
R1 vdd out 50kOhm
M1 out in 0 0 nm l=90nm w=1um
Vin in 0 DC 0.3V AC 1
.model nm nmos level=14 version=4.8.1   

.end

This is a circuit netlist, optimize this circuit with a gain above 20dB and bandwidth above 1Mag Hz.
'''

You should answer:
type_question: '''Analysis this netlist and tell me the type of the circuit ''' 
node_question: '''This is a Spice netlist. Tell me the input and output node name. And the input voltage source name.'''
sim_question: '''This is a Spice netlist for a circuit. Simulate it and give me the ac gain, transirnt gain and bandwidth.''' this should include all the name of specs.
sizing_question: '''Modify the parameter in the netlist. I want the gain at 20dB and bandwidth at 1Mag Hz...''' this should include all the target performance of specs.

Please return a json format dictionary structured with a single key named "questions" with no premable or explanation, which maps to a list. Please do not output 'json' and any symbols such as ''' .
This list contains multiple dictionaries, each representing a specific question. Each dictionary within the list has a key named by the 
name of the question and the content of the key is the content of the question in question: content format. """
tasks_generation_SYSTEM_PROMPT = tasks_generation_template

In [10]:
#task generation
tasks_generation_prompt = f'''system: {tasks_generation_SYSTEM_PROMPT},
"human": "Question: {tasks_generation_question},
Netlist: {netlist}"
'''
#print(tasks_generation_prompt)
tasks = make_chat_completion_request(tasks_generation_prompt)
#print(tasks)

Making ChatCompletion request with streaming enabled...
Streaming response:
{"questions":[{"type_question":"Analyze this netlist and tell me the type of the circuit."},{"node_question":"This is a Spice netlist. Tell me the input and output node names, and identify the input differential stimulus source name."},{"sim_question":"This is a Spice netlist for a circuit. Simulate it and provide: output swing, input offset voltage, input common-mode range, differential ac gain, transient gain, unity-gain bandwidth, phase margin, total power consumption, CMRR and THD."},{"sizing_question":"Modify the device sizes, bias voltages and any passive components in the netlist so that the circuit meets simultaneously: output swing ≥ 1.2 Vpp, input offset ≤ 1 mV, input common-mode range ≥ 1.2 V, ac gain ≥ 65 dB, transient gain ≥ 65 dB, unity-gain bandwidth ≥ 5 MHz, phase margin ≥ 45 °, total power ≤ 50 mW, CMRR ≥ 100 dB, THD ≤ −26 dB."}]}

In [11]:
from eesizer.llm.planner import get_tasks
# get_tasks is now implemented in eesizer.llm.planner and imported here for clarity

Cleaned JSON string:
{"questions":[{"type_question":"Analyze this netlist and tell me the type of the circuit."},{"node_question":"This is a Spice netlist. Tell me the input and output node names, and identify the input differential stimulus source name."},{"sim_question":"This is a Spice netlist for a circuit. Simulate it and provide: output swing, input offset voltage, input common-mode range, differential ac gain, transient gain, unity-gain bandwidth, phase margin, total power consumption, CMRR and THD."},{"sizing_question":"Modify the device sizes, bias voltages and any passive components in the netlist so that the circuit meets simultaneously: output swing ≥ 1.2 Vpp, input offset ≤ 1 mV, input common-mode range ≥ 1.2 V, ac gain ≥ 65 dB, transient gain ≥ 65 dB, unity-gain bandwidth ≥ 5 MHz, phase margin ≥ 45 °, total power ≤ 50 mW, CMRR ≥ 100 dB, THD ≤ −26 dB."}]}


In [12]:
target_value_question = tasks_generation_question
target_value_SYSTEM_PROMPT = ''' 
You are required to extract target circuit performance values from user input.  

Example:  
User input:  
    "This is a circuit netlist, optimize this circuit with a gain above 30dB and bandwidth above 10MHz."  
Expected output:  
{
  "target_values": [
    {
      "gain_target": "30dB",
      "bandwidth_target": "10000000Hz"
    }
  ]
}

Instructions:
- Always return valid JSON only, with no preamble or explanation.  
- Use the structure: {"target_values": [ { performance_target: value, ... } ]}.  
- Preserve any + or - sign in the values.  
- Convert frequency units (e.g., MHz → Hz, kHz → Hz).  
- Each dictionary in the list corresponds to one question.
'''
target_value_prompt = f"""
system: {target_value_SYSTEM_PROMPT}
human: 
Question: {target_value_question}
"""
target_values = make_chat_completion_request(target_value_prompt)


Making ChatCompletion request with streaming enabled...
Streaming response:
{"target_values":[{"output_swing_target":"1.2V","input_offset_target":"0.001V","input_common_mode_range_target":"1.2","ac_gain_target":"65dB","transient_gain_target":"65dB","unity_bandwidth_target":"5000000Hz","phase_margin_target":"45degree","power_target":"0.05W","cmrr_target":"100dB","thd_target":"-26dB"}]}

In [13]:
type_identify_template = ''' 
You are an expert at analogue circuit design. You are required to identify the circuit type from user netlist input. '''
type_identify_prompt = f''' System: {type_identify_template}
Netlist: {netlist},
'''
print(type_identify_prompt)

 System:  
You are an expert at analogue circuit design. You are required to identify the circuit type from user netlist input. 
Netlist:  
.title Basic amp
.include 'ptm_90.txt'
.param Vcm = 0.6
M3 d3 in2 midp vdd pmos W=1u L=90n
M4 d4 in1 midp vdd pmos W=1u L=90n
M6 midp bias1 vdd vdd pmos W=1u L=90n

M1 d1 in1 midn 0 nmos W=1u L=90n
M2 d2 in2 midn 0 nmos W=1u L=90n
M5 midn bias2 0 0 nmos W=1u L=90n

M7 d1 g7 vdd vdd pmos W=1u L=90n
M8 d2 g7 vdd vdd pmos W=1u L=90n
M9 g7 bias3 d1 vdd pmos W=1u L=90n
M10 d10 bias3 d2 vdd pmos W=1u L=90n

M15 g7 bias5 s15 0 nmos W=1u L=90n
M16 s15 bias6 g7 vdd pmos W=1u L=90n
M17 d10 bias5 s17 0 nmos W=1u L=90n
M18 s17 bias6 d10 vdd pmos W=1u L=90n

M11 s15 bias4 d4 0 nmos W=1u L=90n
M12 s17 bias4 d3 0 nmos W=1u L=90n
M13 d4 s15 0 0 nmos W=1u L=90n
M14 d3 s15 0 0 nmos W=1u L=90n

M19 out d10 vdd vdd pmos W=1u L=90n
M20 out s17 0 0 nmos W=1u L=90n

Cc1 out d10 5p
Cc2 out s17 5p
Cl out 0 10p 
Rl out 0 1k

vbias1 bias1 0 DC 0.6
vbias2 bias2 0 DC 0.6
vbias

In [14]:
type_identified = make_chat_completion_request(type_identify_prompt)
print(type_identified)

Making ChatCompletion request with streaming enabled...
Streaming response:
The netlist is an integrated-CMOS operational amplifier.

Key observations  
• Complementary input stage:  
  – M1/M2 (NMOS) and M3/M4 (PMOS) form two concurrent differential pairs → rail-to-rail input capability.  
• Bias transistors M5, M6 supply the tail currents for the two pairs.  
• Nodes d1/d2 and d3/d4 are routed through cascode / mirror devices (M7–M14, M15–M18) that constitute the second high-gain stage.  
• Output stage: M19 (PMOS) and M20 (NMOS) form a push-pull class-AB output driver.  
• Frequency compensation: Cc1 and Cc2 provide Miller / nested-Miller compensation between the output node “out” and the internal second-stage nodes (d10, s17).  
• Load elements Cl and Rl model the external load of an op-amp.

Hence, the whole structure is a three-stage, rail-to-rail input / class-AB output CMOS operational amplifier (RRIO op-amp). The netlist is an integrated-CMOS operational amplifier.

Key observ

In [15]:
from eesizer.llm.planner import nodes_extract
# nodes_extract imported from eesizer.llm.planner (robust node extractor)

def extract_code(text):
    regex1 = r"'''(.+?)'''" 
    regex2 = r"```(.+?)```"

    matches1 = re.findall(regex1, text, re.DOTALL)
    matches2 = re.findall(regex2, text, re.DOTALL)

    extracted_code = "\n".join(matches1 + matches2)
    lines = extracted_code.split('\n')
    cleaned_lines = []

    for line in lines:
        if '*' in line:
            line = line.split('*')[0].strip()
        elif '#' in line:
            line = line.split('#')[0].strip()
        elif ';' in line:
            line = line.split(';')[0].strip()
        elif line.startswith('verilog'):
            line = '\n'
        if line:  
            cleaned_lines.append(line)
    
    cleaned_code = "\n".join(cleaned_lines)
    return cleaned_code

node_SYSTEM_PROMPT = '''
You aim to find the circuit output nodes, input nodes and input voltage source name from the given netlist. 
Here is an example output:
{"nodes": [{"input_node": "in"},
                {"output_node": "out"},        
                {"source_name": "Vin"}]}
Please do no add any description in the output, just the dictionary.
'''

node_prompt = f""" 
system: {node_SYSTEM_PROMPT},
human: Question: {node_question}
Netlist: {netlist}
"""

In [16]:
nodes = make_chat_completion_request(node_prompt)
print(nodes)

Making ChatCompletion request with streaming enabled...
Streaming response:
{"nodes":[{"input_node":"in1"},{"input_node":"in2"},{"output_node":"out"},{"source_name":"Vid"}]} {"nodes":[{"input_node":"in1"},{"input_node":"in2"},{"output_node":"out"},{"source_name":"Vid"}]}


In [17]:
input_nodes, output_nodes, source_names = nodes_extract(nodes)
print("----------------------node extract-----------------------------")
print(f"input_nodes:{input_nodes}")
print(f"output_nodes:{output_nodes}")
print(f"source_names:{source_names}")

----------------------node extract-----------------------------
input_nodes:['in1', 'in2']
output_nodes:['out']
source_names:['Vid']


In [18]:
#fuctions for different simulation type
r

In [ ]:
# Deprecated shim removed — use vgscheck_from_oplog_file
# (This cell previously defined filter_lines; behavior now delegated to the parser-backed helper.)


In [ ]:
# Deprecated shim removed — use vgscheck_from_oplog_file
# (This cell previously defined convert_to_csv; behavior now delegated to the parser-backed helper.)


In [ ]:
# Deprecated shim removed — use vgscheck_from_oplog_file
# (This cell previously defined format_csv_to_key_value; behavior now delegated to the parser-backed helper.)


In [ ]:
# Replaced CSV pipeline with canonical parser-backed call
vgscheck_from_oplog_file(input_txt, output_txt)


In [ ]:
# Replaced CSV pipeline with canonical parser-backed call
vgscheck_from_oplog_file(input_txt, output_txt)


In [ ]:
# Ensure any later/redefined notebook functions delegate to the package Orchestrator
from typing import Dict, Any, Optional, List
from eesizer.agents.orchestrator import Orchestrator
import os

# Backward-compatible tool_calling for older notebook cells that expect different signatures.
def tool_calling(tool_chain: Dict[str, Any], netlist_text: Optional[str] = None, run_dir: Optional[str] = None) -> Dict[str, Any]:
    """Flexible wrapper that uses Orchestrator.

    Behavior:
    - If tool_chain contains a key 'netlist_variants' (list of netlist strings), call optimize().
    - Else if netlist_text is provided, call run_once() for a single evaluation.
    - Else attempt to use a global 'netlist' variable if present.
    """
    orch = Orchestrator(run_dir=run_dir)

    # variant optimization path
    variants = None
    if isinstance(tool_chain, dict) and 'netlist_variants' in tool_chain:
        variants = tool_chain.get('netlist_variants')
    if variants:
        return orch.optimize(variants, tool_chain, run_dir_base=run_dir)

    # single-run path
    nl = netlist_text or globals().get('netlist')
    if nl:
        return orch.run_once(nl, tool_chain)

    return {"success": False, "error": "no netlist provided"}


def optimization(netlist_variants: List[str], tool_chain: Dict[str, Any], run_dir: Optional[str] = None) -> Dict[str, Any]:
    orch = Orchestrator(run_dir=run_dir)
    return orch.optimize(netlist_variants, tool_chain, run_dir_base=run_dir)
